In [9]:
from flask import Flask, request, jsonify
import pandas as pd
import joblib
import os

app = Flask(__name__)

# Cargar el modelo al inicio de la aplicación
clf = joblib.load(r'C:\Users\jacosta\Documents\GitHub\Mis Repos\MIAD_NLP_2024\Semana 4\proyecto\stacking_model.pkl')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Obtener datos de la solicitud como JSON
        data = request.get_json()

        # Convertir a DataFrame
        input_data = pd.DataFrame([data])

        # Realizar predicción
        prediction = clf.predict(input_data)

        # Devolver resultado como JSON
        return jsonify({'prediction': prediction.tolist()}), 200
    except Exception as e:
        return jsonify({'error': str(e)}), 400

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1

c:\Users\jacosta\AppData\Local\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [9]:
from flask import Flask
from flask_restx import Api, Resource, fields, reqparse
from flask_cors import CORS
import requests

app = Flask(__name__)
CORS(app)  # Habilitar CORS para todas las rutas y orígenes

api = Api(
    app, 
    version='1.0', 
    title='Predictor de Precios de Autos',
    description='API para predecir precios de autos')

ns = api.namespace('predict', 
     description='Predictor de Precio')

parser = reqparse.RequestParser()
parser.add_argument(
    'Year', 
    type=int, 
    required=True, 
    help='Año del automóvil', 
    location='args')
parser.add_argument(
    'Mileage', 
    type=int, 
    required=True, 
    help='Millaje del automóvil', 
    location='args')
parser.add_argument(
    'State', 
    type=int, 
    required=True, 
    help='Estado del automóvil', 
    location='args')
parser.add_argument(
    'Make', 
    type=int, 
    required=True, 
    help='Marca del automóvil', 
    location='args')
parser.add_argument(
    'Model', 
    type=int, 
    required=True, 
    help='Modelo del automóvil', 
    location='args')

resource_fields = api.model('Resource', {
    'result': fields.Float,
})

@ns.route('/')
class CarPricePrediction(Resource):

    @api.doc(parser=parser)
    @api.marshal_with(resource_fields)
    def get(self):
        args = parser.parse_args()
        
        year = args['Year']
        mileage = args['Mileage']
        state = args['State']
        make = args['Make']
        model = args['Model']
        
        # Hacer una solicitud POST al servicio de predicción
        prediction_service_url = "http://localhost:5000/predict"
        data = {
            'Year': year,
            'Mileage': mileage,
            'State': state,
            'Make': make,
            'Model': model
        }
        response = requests.post(prediction_service_url, json=data)
        
        if response.status_code == 200:
            result = response.json()['prediction']
            return {'result': result}, 200
        else:
            return {'error': 'No se pudo realizar la predicción'}, 500
    
if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, host='0.0.0.0', port=5000)




 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.181:5000
Press CTRL+C to quit
127.0.0.1 - - [29/Apr/2024 03:56:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2024 03:56:58] "GET /swaggerui/droid-sans.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 03:56:58] "GET /swaggerui/swagger-ui.css HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 03:56:58] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 03:56:58] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Apr/2024 03:56:58] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2024 03:57:17] "POST /predict HTTP/1.1" 404 -
127.0.0.1 - - [29/Apr/2024 03:57:17] "GET /predict/?Year=2014&Mileage=31009&State=20&Make=27&Model=305 HTTP/1.1" 500 -
127.0.0.1 - - [29/Apr/2024 03:57:39] "POST /predict HTTP/1.1" 404 -
127.0.0.1 - - [29/Apr/2024 03:57:39] "GET /predict/?Year=2014&Mileage=31009&State=20&Make=27&Model=305 HTTP/1.1" 500 -
